<a href="https://colab.research.google.com/github/john-decker/Dutch_Colonial_Research/blob/main/Scripts/Simple_DB_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Working with the Database
In the previous script, we created a working database, checked it, and saved it. Now, we will begin by loading the database into Colab's file manager so that it is available to us for queries.

We must import SQLLite and reestablish a connection.

In [1]:
import sqlite3

db_path = '/content/Dutch_Colonial_DB'

conn = sqlite3.connect(db_path)


###Making Sure the Connection Works
Now that we have a connection, we will create a cursor object and rerun the sample query we used to ensure that the database was properly working in the previous script.

In [2]:
cursor = conn.cursor()

cursor.execute("SELECT * FROM Person")
results = cursor.fetchall()

for result in results:
    print(result)

conn.commit()
conn.close()

('1', 'Delavall', '', 'M', '2', '1')
('2', 'Ten Houdt', 'Severyn', 'M', '1', '1')
('3', 'DuBooys (Dubois)', 'Lowies (Louys)', 'M', '1', '1')
('4', 'Blansjan', 'Matthias', 'M', '1', '1')
('5', 'Fortune', 'Jan', 'M', '3', '1')
('6', 'Lavall', '', 'M', '1', '1')
('7', 'Ackerman', 'Lodowyck', 'M', '1', '1')
('8', 'DeGraef', 'Moses', 'M', '1', '1')
('9', 'Fisher', 'William', 'M', '1', '1')
('10', 'Haegen', 'Bruyn', 'M', '1', '1')
('11', 'Claesen DeWitt', 'Tierk', 'M', '1', '1')
('12', 'Osterhoudt', 'Jan', 'M', '1', '1')
('13', 'Tynhoudt', 'Cronelis', 'M', '1', '1')
('14', 'Lowersen', 'Jan', 'M', '1', '1')
('15', 'Cool', 'Pieter', 'M', '1', '1')
('16', 'Cool', 'Leendert', 'M', '1', '1')
('17', 'Unnamed', '', 'M', '3', '1')
('18', 'Jansen', 'Dirck', 'M', '1', '1')
('19', 'Grevenraedt', '', 'M', '1', '4')
('20', 'Martensen', 'Aerdt', 'M', '1', '1')
('21', 'Addesen', 'Anthony', 'M', '1', '1')
('22', 'Joshensen', 'Hendrick', 'M', '1', '1')
('23', 'Wittikar', 'Eduward', 'M', '1', '1')
('24', 'Wyn

###Multiple Join Operations
The data for our model is broken across multiple entities. In order to relate them, we can carry out join operations across multiple tables. This will allow us, for example, to associate particular people in the Person table with particular cases from the Session_Details table and what their role (e.g. defendant, complainant) was from the Participation_Type table.

In [3]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''SELECT firstName, lastName, roleDescription, minutesText
FROM Person
INNER JOIN Person_to_Session
ON Person.personID = Person_to_Session.personID
INNER JOIN Session_Details
ON Person_to_Session.sessionDetailsID = Session_Details.sessionDetailsID
INNER JOIN Participation_Type
ON Person_to_Session.participationID = Participation_Type.participationID;''')

results = cursor.fetchall()

for result in results:
    if result[0] and not result[1]:
      fullName = f'{result[0]}'
    if not result[0] and result[1]:
      fullName = f'{result[1]}'
    else:
      fullName = f'{result[0]} {result[1]}'

    print(f'Name: {fullName}\nRole: {result[2]}\nCase Details: {result[3]}\n')

conn.commit()
conn.close()

Name: Delavall
Role: complainant, criminal
Case Details: Full Minutes under Blansjan's arms.  Further says that when Anthony Crispel had been caught [(was yervangen) also means: was a prisoner] he went to Capt. Delavall to hear what Capt. Dlavall was saying [and] for the purpose of speaking about the affairs, but there was no opportunity for speaking of it.  Then Severeyn arrived there and said: "I gave the old Walloon a blow before the head so that he tumbled down." Then DuPue answered: "You ought not to have done that." Sueveryn Ten houdt says: "At the time of Anthony Crispel's detention in the guard house, Blansjan hit him on the chest, and Blansjan twice said: "What are you doing here, you are Laval's informer." Then he hit Matthias Blansjan on the head." Dirck Jansen, having been sworn, declares that Blans- jan pushed Sueryn Ten Houdt away from the door, and heard some words about "informer".(verklicker). Pieter Jansen, having been sworn, declares that Lowies Du Booys was busy bre

Add the Session_Action table to pull out different information

In [4]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''SELECT firstName, lastName, roleDescription, description, amount
FROM Person
INNER JOIN Person_to_Session
ON Person.personID = Person_to_Session.personID
INNER JOIN Session_Details
ON Person_to_Session.sessionDetailsID = Session_Details.sessionDetailsID
INNER JOIN Participation_Type
ON Person_to_Session.participationID = Participation_Type.participationID
INNER JOIN Session_Action
ON Session_Details.sessionDetailsID = Session_Action.sessionDetailsID;''')

results = cursor.fetchall()

for result in results:
    if result[0] and not result[1]:
      fullName = f'{result[0]}'
    if not result[0] and result[1]:
      fullName = f'{result[1]}'
    else:
      fullName = f'{result[0]} {result[1]}'

    print(f'Name: {fullName}\nRole: {result[2]}\nAction: {result[3]}\nAmount: {result[4]}\n')

conn.commit()
conn.close()

Name: Delavall
Role: complainant, criminal
Action: Fine
Amount: 6 Sch. Wheat per defendant

Name: Severyn Ten Houdt
Role: defendant, criminal
Action: Fine
Amount: 6 Sch. Wheat per defendant

Name: Severyn Ten Houdt
Role: sale of enslaved person participant
Action: Sale of a Black enslaved man
Amount: 1000 Gldrs.

Name: Lowies (Louys) DuBooys (Dubois)
Role: defendant, criminal
Action: Fine
Amount: 6 Sch. Wheat per defendant

Name: Lowies (Louys) DuBooys (Dubois)
Role: sale of enslaved person participant
Action: Sale of a Black enslaved man
Amount: 1000 Gldrs.

Name: Lowies (Louys) DuBooys (Dubois)
Role: respondent, civil
Action: Fine
Amount: 80 Gldrs.

Name: Lowies (Louys) DuBooys (Dubois)
Role: sale of enslaved person participant
Action: Sale of a Black enslaved woman and a Black enslaved man
Amount: 800 Gldrs.

Name: Matthias Blansjan
Role: defendant, criminal
Action: Fine
Amount: 6 Sch. Wheat per defendant

Name: Matthias Blansjan
Role: sale of enslaved person participant
Action: Sal

If we want to limit output to criminal defendants or civil respondents, we can filter using WHERE

In [5]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''SELECT firstName, lastName, gender, ethnicityDescription, statusDescription, roleDescription, description, amount
FROM Person
INNER JOIN Ethnicity
ON Person.ethnicityID = Ethnicity.ethnicityID
INNER JOIN Status
ON Person.statusID = Status.statusID
INNER JOIN Person_to_Session
ON Person.personID = Person_to_Session.personID
INNER JOIN Session_Details
ON Person_to_Session.sessionDetailsID = Session_Details.sessionDetailsID
INNER JOIN Participation_Type
ON Person_to_Session.participationID = Participation_Type.participationID
INNER JOIN Session_Action
ON Session_Details.sessionDetailsID = Session_Action.sessionDetailsID
WHERE roleDescription LIKE '%defendant%' OR roleDescription LIKE '%respondent%';''')

results = cursor.fetchall()

for result in results[8:]: #limit output to the first 8 records
    if result[0] and not result[1]:
      fullName = f'{result[0]}'
    if not result[0] and result[1]:
      fullName = f'{result[1]}'
    else:
      fullName = f'{result[0]} {result[1]}'

    print(f'Name: {fullName}\nGender: {result[2]}\nEtnicity: {result[3]}\nStatus: {result[4]}\nRole: {result[5]}\nAction: {result[6]}\nAmount: {result[7]}\n')

conn.commit()
conn.close()

Name: Lowies (Louys) DuBooys (Dubois)
Gender: M
Etnicity: European
Status: Colonist
Role: respondent, civil
Action: Fine
Amount: 80 Gldrs.

Name: Altjen Sybrands
Gender: F
Etnicity: European
Status: Colonist
Role: respondent, civil
Action: Complainant demands vindication of his honor
Amount: None

Name: Mathys Roeloofsen
Gender: M
Etnicity: European
Status: Colonist
Role: respondent, civil
Action: Complainant demands vindication of his honor
Amount: None

Name: Annetjen Aerts
Gender: F
Etnicity: European
Status: Colonist
Role: respondent, civil
Action: Complainant demands vindication of his honor, Respondent demands vindication of her honor
Amount: None

Name: Moses DeGraef
Gender: M
Etnicity: European
Status: Colonist
Role: respondent, civil
Action: Fine
Amount: 13 Sch. Wheat

Name: Lodowyck Ackerman
Gender: M
Etnicity: European
Status: Colonist
Role: respondent, civil
Action: Fine
Amount: 8 Gldrs.

Name: Gysbert Albertsen
Gender: M
Etnicity: European
Status: Colonist
Role: respondent

##Creating a new .csv file from the results of a query
Now that we've stitched together various tables to get useful information, we can create a query, return results, and then write those results to a .csv that we can download and use for whatever purposes we need.

In [6]:
import csv #import the functionality needed to work with csv files
headers = ['fname', 'lname', 'gender', 'ethnicity', 'status', 'role', 'action', 'amount']

with open('defendants.csv', 'a') as csv_object:
  writer= csv.DictWriter(csv_object, fieldnames = headers)
  writer.writeheader()

  for entry in results:
    row = {'fname': entry[0], 'lname': entry[1], 'gender': entry[2], 'ethnicity': entry[3],'status': entry[4], 'role': entry[5], 'action': entry[6], 'amount': entry[7]}
    writer.writerow(row)

print('file written')


file written


In [7]:
#Another CSV for later use
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''SELECT personID, firstName, lastName, gender, ethnicityDescription, statusDescription
FROM Person
LEFT JOIN Ethnicity
ON Person.ethnicityID = Ethnicity.ethnicityID
LEFT JOIN Status
ON Person.statusID = Status.statusID;''')

person_results = cursor.fetchall()

person_headers = ['personid', 'fname', 'lname', 'gender', 'ethnicity', 'status']

with open('person_all.csv', 'a') as csv_object:
  writer= csv.DictWriter(csv_object, fieldnames = person_headers)
  writer.writeheader()

  for entry in person_results:
    row = {'personid': entry[0],'fname': entry[1], 'lname': entry[2], 'gender': entry[3], 'ethnicity': entry[4],'status': entry[5]}
    writer.writerow(row)

print('file written')


conn.commit()
conn.close()

file written


##Trying to Isolate Potential Numeric Information
The sums recorded for fines in the database are strings (text) rather than numbers. This makes sense because the number is associated with a currency type (denominated in wheat, Guilders, etc.) It would be nice to be able to isolate the numbers to make it possible to get counts of things.


In [8]:
#enumerate the results to get a sense of the internal structure

for result in enumerate(results):
  print(result)

(0, ('Severyn', 'Ten Houdt', 'M', 'European', 'Colonist', 'defendant, criminal', 'Fine', '6 Sch. Wheat per defendant'))
(1, ('Lowies (Louys)', 'DuBooys (Dubois)', 'M', 'European', 'Colonist', 'defendant, criminal', 'Fine', '6 Sch. Wheat per defendant'))
(2, ('Matthias', 'Blansjan', 'M', 'European', 'Colonist', 'defendant, criminal', 'Fine', '6 Sch. Wheat per defendant'))
(3, ('Leendert', 'Cool', 'M', 'European', 'Colonist', 'respondent, civil', 'Fine', '28 Sch. Wheat, plus costs'))
(4, ('Dirck', 'Jansen', 'M', 'European', 'Colonist', 'respondent, civil', 'Attempt to resolve payment dispute', 'None'))
(5, ('Dirck', 'Jansen', 'M', 'European', 'Colonist', 'respondent, civil', 'Fine', '25 Gldrs.'))
(6, ('Anthony', 'Addesen', 'M', 'European', 'Colonist', 'respondent, civil', 'Change of jurisdiction', 'None'))
(7, ('Eduward', 'Wittikar', 'M', 'European', 'Colonist', 'respondent, civil', 'Court orders payment of fines', 'Unspecified'))
(8, ('Lowies (Louys)', 'DuBooys (Dubois)', 'M', 'European

In [9]:
#isolate a single portion of a single entry targeting where fine amount information is (if any)
print(results[14][7])

315 Gldrs, 10 St., plus costs


One way of approaching the problem is to use regular expressions to isolate the numbers from the string.

In [10]:
#use regular expressions to isolate potential numbers from the string
#NOTE: \b = word boundary, \d = digit from 0-9, + = greedy search (one or more)
import re
print([int(number) for number in re.findall(r'\b\d+\b', results[14][7])])

[315, 10]


This works but we lose important information regarding the denomination that the number referst to (e.g. Guilders and Stuivers).

Another approach might include parsing the strings and checking for certain conditions. Having some knowledge of the fine denominations so far, we can create queries to isolate each type and then parse the results to get specific numbers. To do this, we will designate a list of terms we want to search for. This will allow us to add more terms as we encounter other types of currency (e.g. wampum, beaver pelt, etc.)

In [11]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

search_terms = ['Gldrs', 'Sch']

for term in search_terms:
  cursor.execute(f"SELECT * FROM Session_Action WHERE amount LIKE '%{term}%';")

  currency_results = cursor.fetchall()

  for result in currency_results:
    amount_field = result[3].split() #use the string .split() method to make a list out of the string
    # create a test for cases where there are guilders and stuivers to make sure we catch both
    if len(amount_field) > 3 and amount_field[3] == 'St.,':
       print(f'{int(amount_field[0])} {amount_field[1]} {int(amount_field[2])} {amount_field[3]}')
    else:
      print(f'{int(amount_field[0])} {amount_field[1]}')

  conn.commit()


conn.close()

80 Gldrs.
8 Gldrs.
315 Gldrs, 10 St.,
176 Gldrs.,
81 Gldrs.,
25 Gldrs.
1000 Gldrs.
800 Gldrs.
6 Sch.
13 Sch.
24 Sch.
20 Sch.
28 Sch.


Now that we've tested the idea, let's do some work with it. Let's get a total for the amounts of each currency type to see how much was assigned as fines for the entires we have.

In [12]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

search_terms = ['Gldrs', 'Sch']

#initialize a dictionary to keep count of the total of each currency type
results_dict = {'Gldrs': 0,
                'St': 0,
                'Sch': 0}

for term in search_terms:
  cursor.execute(f"SELECT * FROM Session_Action WHERE amount LIKE '%{term}%';")

  currency_results = cursor.fetchall()

  for result in currency_results:
    amount_field = result[3].split()
    if len(amount_field) > 3 and amount_field[3] == 'St.,':
      results_dict[term] += int(amount_field[0])
      results_dict['St'] += int(amount_field[2])

    else:
      results_dict[term] += int(amount_field[0])


  conn.commit()

conn.close()

print(results_dict)

{'Gldrs': 2485, 'St': 10, 'Sch': 91}
